In [12]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as data
import datetime

russell_index = ["^RUI"]

portfolio = ["AMZN", "MSFT", "GOOG"]
assets = np.array([10000.0, 1000.0, 1000.0]) 
exposures = np.zeros(3)


#Set timeframe
start = "2015-12-07"
end = "2019-03-31"


#Pull data from Yahoo! Finance
df = data.DataReader(russell_index, data_source="yahoo", start=start, end=end)

print(df["Adj Close"])

for i, ticker in enumerate(portfolio):
    df = data.DataReader(portfolio, data_source="yahoo", start=start, end=end)

    price = df["Adj Close"]
    # Assign the exposure at each index to the price times the allocation at each index
    exposures[i] = price * assets[i] 
    # Parse data
    
    print(ticker+" Exposure in USD: ${:,.2f}".format(exposures[i]))
    







Symbols            ^RUI
Date                   
2015-12-07  1150.430054
2015-12-08  1143.050049
2015-12-09  1133.959961
2015-12-10  1136.709961
2015-12-11  1114.630005
2015-12-14  1118.630005
2015-12-15  1130.709961
2015-12-16  1147.109985
2015-12-17  1130.050049
2015-12-18  1110.739990
2015-12-21  1118.989990
2015-12-22  1128.750000
2015-12-23  1143.050049
2015-12-24  1141.449951
2015-12-28  1138.790039
2015-12-29  1150.469971
2015-12-30  1142.130005
2015-12-31  1131.880005
2016-01-04  1114.859985
2016-01-05  1116.839966
2016-01-06  1101.810059
2016-01-07  1075.290039
2016-01-08  1063.550049
2016-01-11  1063.640015
2016-01-12  1071.280029
2016-01-13  1043.969971
2016-01-14  1060.609985
2016-01-15  1038.030029
2016-01-19  1037.410034
2016-01-20  1025.680054
...                 ...
2019-02-19  1542.660034
2019-02-20  1545.140015
2019-02-21  1539.670044
2019-02-22  1549.969971
2019-02-25  1552.000000
2019-02-26  1550.130005
2019-02-27  1550.290039
2019-02-28  1545.729980
2019-03-01  1556

ValueError: setting an array element with a sequence.